<img src="../static/logopython.png" alt="Logo Python" style="width: 300px; display: inline"/>
<img src="../static/deimoslogo.png" alt="Logo Deimos" style="width: 300px; display: inline"/>

# Clase 7: Taller de depuración y pruebas

En este capítulo vamos a hacer un workshop. Simularemos una sesión de testing y depuración usando las herramientas que Python pone a nuestra disposición:

* El [módulo unittest](https://docs.python.org/3.5/library/unittest.html) y su interfaz de línea de comandos
* El [módulo logging](https://docs.python.org/3.5/library/logging.html) para mostrar mensajes de error
* El [depurador de Python, pdb](https://docs.python.org/3.5/library/pdb.html)

Vamos a utilizar el código disponible en [este repositorio de Github](https://github.com/jorgeas80/nicar2016-python-testing-debugging-exercises). Se puede seguir el taller de dos maneras:

* Descargando el código del repositorio en formato zip en nuestra máquina
* Clonando el repositorio en nuestra máquina

<div class="alert alert-info">Si clonamos el repositorio en nuestra máquina, podemos ir alternando entre la rama *master*, en la que trabajaremos, y la rama *solutions*, donde se encuentran resueltos los ejercicios propuestos durante el taller</div>

## Tests unitarios

Vamos a empezar ejecutando unos cuantos tests unitarios

```python
python3 -m unittest tests.test_basic```

Veremos una salida como ésta

```bash
t_basic
F..
======================================================================
FAIL: test_true_is_true (tests.test_basic.FailingTestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/jorge/Dev/github/nicar2016-python-testing-debugging-exercises/tests/test_basic.py", line 13, in test_true_is_true
    self.assertEqual(False, True)
AssertionError: False != True

----------------------------------------------------------------------
Ran 3 tests in 0.001s

FAILED (failures=1)```

Abriendo el fichero tests/test_basic.py, vemos el test que ha fallado. Es bastante sencillo

```python3
self.assertEqual(False, True)```


En este punto, aun no hemos visto nada sobre tests unitarios en Python, pero mirando el código, entendemos lo básico:

* El módulo que importamos se llama *unittest*. Contiene lo necesario para implementar tests unitarios
* La clase básica es *TestCase*. De ahí heredaremos las nuestras
* Dentro de cada clase hija de *TestCase*, definiremos tantos métodos como pruebas queramos hacer
* Compararemos los valores esperados con los valores devueltos mediante la familia de funciones *assert*

Continuamos lanzando tests. Dentro una test suite podemos lanzar tests cases individuales

```python

python3 -m unittest tests.test_basic.NoFailuresTestCase```

Incluso funciones de tests individuales

```python 

python3 -m unittest tests.test_basic.NoFailuresTestCase.test_true_is_true```

### Capturando excepciones en tests

Es posible que se nos haya escapado capturar alguna excepción, y los tests ni siquiera lleguen a ejecutarse por eso. Para provocar ese error, ejecutar:

```python

python3 -m unittest tests.test_result_loader```

Obtendremos algo así

```bash
t_result_loader
E
======================================================================
ERROR: test_load_bad_json (tests.test_result_loader.SimpleResultLoaderTestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/jorge/Dev/github/nicar2016-python-testing-debugging-exercises/tests/test_result_loader.py", line 13, in test_load_bad_json
    results = loader.load(sample_json)
  File "/home/jorge/Dev/github/nicar2016-python-testing-debugging-exercises/results/__init__.py", line 11, in load
    parsed = json.loads(s)
  File "/usr/lib/python3.5/json/__init__.py", line 319, in loads
    return _default_decoder.decode(s)
  File "/usr/lib/python3.5/json/decoder.py", line 339, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/usr/lib/python3.5/json/decoder.py", line 357, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 4 column 9 (char 42)

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)```

Como vemos, el error está en el fichero *tests/test_result_loader.py*. Concretamente en la línea 13, función *test_load_bad_json*

Abrimos el fichero, y vemos que en esa línea se llama a una función que carga un dato de tipo JSON, pero dentro de la función, no se comprueba si el dato ha sido correctamente cargado o no antes de operar con él. Eso provoca un error al intentar acceder a uno de sus campos

<div class="alert alert-warning">__Ejercicio__: Arregla la situación para evitar que se produzca la excepción incontrolada</div>

Esto tipo de problemas casi siempre se da porque damos por sentadas cosas sin probarlas antes. Por ejemplo, ¿qué pruebas ejecutarías en este trozo de código para tener controlados los errores?

```python
def pruebame(items):
    return [items[1].upper(), items[0].upper()]```

<div class="alert alert-warning">__Ejercicio__: Haz la función anterior segura ante potenciales errores por inconsistencia en los datos de entrada

### TDD

Un posible enfoque a la hora de desarrollar software es escribir primero el código de prueba, y luego el código a probar, una vez has cubierto todos los casos que se necesitan. Sus defensores mantienen que ayuda a escribir mejor código y más mantenible.

Vamos a intentar este enfoque con un ejercicio sencillo:

El fichero *tests/test_names.py* está vacío. Crea una clase de tests llamada TestNames. Dentro, un solo método, llamado *test_parse_names*. En dicho método, llamarás a una función *parse_names*, que implementarás dentro del fichero *names/\__init\__.py*. 

La función recibirá como argumento una cadena con las partes del nombre de una persona, separadas por comas. Por ejemplo: 

*Arévalo, de Soto, Jorge, Sr.*. 

Como respuesta, devolverá un json como éste:

```json
{
    "first_name": "Jorge",
    "last_name": "de Soto",
    "middle_name": "Arévalo",
    "suffix": "Sr"
}```

Si el sufijo no existe, se devolverá *None* en ese campo.

Por supuesto, habrás de implementar luego la clase *parse_names*, importarla, y llamarla con los casos de prueba que se te hayan ocurrido. 

Una vez termines tanto el código de tests como la función en si, podrás ejecutar el test de esta forma

```python
python3 -m unittest tests.test_names```

Y deberías obtener una salida similar a ésta

```bash
test_names
.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK```

<div class="alert alert-warning">__Ejercicio__: Implementa la clase de pruebas y la función a probar</div>

### Refactorizando

A veces, diseñamos nuestro código de manera que es complicado de probar. Creamos clases o funciones excesivamente complejas, cuando lo ideal sería crear pequeñas funciones que fueran sencillas de probar.

Por ejemplo, corramos el siguiente test

```python

python3 -m unittest tests.test_chicago_result_loader.TestChicagoResultLoader```

Si abres el código del test, verás que se prueba el método *ChicagoResultsLoader.load*, que abre un fichero de texto y carga datos, devolviendo una estructura de tipo JSON, cuyos valores se comprueban en el test.

Sería más interesante que tuviéramos dos funciones:

* función *load*, para cargar los datos del fichero, línea a línea
* funcion *parse_result*, para transformar la línea leída en un elemento JSON

Y pudiéramos probar esas dos funciones por separado desde nuestra función de test.

Dado que en esas circunstancias pasaríamos a tener más de una función de test, hay ciertas tareas repetitivas que ejecutamos en cada caso de test, y que podríamos extraer fuera (instanciar la clase a probar, y obtener la ruta al fichero de datos).

Por último, podría ser interesante añadir funcionalidad de log al código a probar. De esta forma, además de probarlo con los tests, también generaríamos en un fichero de texto la salida que la función probada va generando

<div class="alert alert-warning">__Ejercicio__: Haz las siguientes tareas:
 
 <ul>
 <li>Refactoriza el código de la clase ChicagoResultsLoader.load, dividiendolo en dos métodos: load y parse_result, que probarás por separado</li>
 <li>En cada caso de test, habrá código repetitivo que podrás sacar fuera (instanciar la clase a probar, y obtener la ruta al fichero de datos). Investiga el uso de la función [unittest.TestCase.setUp](https://docs.python.org/3.5/library/unittest.html#unittest.TestCase.setUp) para centralizar tareas repetitivas que se tendrían que realizar en todos los tests</li>
<li>Añade a los métodos refactorizados ChicagoResultsLoader.load y ChicagoResultsLoader.parse_result funcionalidad de logging. Deberán sacar el log a un fichero de texto externo. Investiga el módulo [logging](https://docs.python.org/3.5/library/logging.html) para averiguar cómo sacar log a un fichero externo</li>
</ul>
</div>


## Depurando código

Por muchos tests que escribamos y muy bien que diseñemos nuestro código, en algún momento tendremos que acabar depurando para encontrar algún error que se nos ha escapado. Vamos a ver el uso básico del depurador de Python, *pdb*, y algunas funciones útiles

Empecemos con algo sencillo. Mostrar el contenido de un objeto JSON en una sesión de depuración. Podemos abrir una consola del intérprete desde la carpeta *tests/data*, o ejecutar la celda de abajo

In [1]:
# Cargamos datos JSON desde fichero
import json
with open('ap_elections_loader_recording-1456935370.json') as f:
    data = json.load(f)
    print(data['races'][0])

{'raceID': '10673', 'party': 'GOP', 'lastUpdated': '2016-03-02T15:42:49Z', 'statePostal': 'FL', 'raceTypeID': 'R', 'national': True, 'officeID': 'P', 'officeName': 'President', 'candidates': [{'first': 'Jeb', 'party': 'GOP', 'last': 'Bush', 'ballotOrder': 1, 'polNum': '14561', 'polID': '1239', 'candidateID': '20408'}, {'first': 'Ben', 'party': 'GOP', 'last': 'Carson', 'ballotOrder': 2, 'polNum': '14562', 'polID': '64509', 'candidateID': '20409'}, {'first': 'Chris', 'party': 'GOP', 'last': 'Christie', 'ballotOrder': 3, 'polNum': '14563', 'polID': '60051', 'candidateID': '20410'}, {'first': 'Ted', 'party': 'GOP', 'last': 'Cruz', 'ballotOrder': 4, 'polNum': '14564', 'polID': '61815', 'candidateID': '20411'}, {'first': 'Carly', 'party': 'GOP', 'last': 'Fiorina', 'ballotOrder': 5, 'polNum': '14566', 'polID': '60339', 'candidateID': '20414'}, {'first': 'Lindsey', 'party': 'GOP', 'last': 'Graham', 'ballotOrder': 7, 'polNum': '14568', 'polID': '1408', 'candidateID': '20416'}, {'abbrv': 'Huckab

Bastante poco práctico. Vamos a usar el módulo [pprint](https://docs.python.org/3/library/pprint.html)

In [5]:
# Ahora usamos pprint para mostrarlo por pantalla
import json
import pprint
with open('ap_elections_loader_recording-1456935370.json') as f:
    data = json.load(f)
    pprint.pprint(data['races'][0])

{'candidates': [{'ballotOrder': 1,
                 'candidateID': '20408',
                 'first': 'Jeb',
                 'last': 'Bush',
                 'party': 'GOP',
                 'polID': '1239',
                 'polNum': '14561'},
                {'ballotOrder': 2,
                 'candidateID': '20409',
                 'first': 'Ben',
                 'last': 'Carson',
                 'party': 'GOP',
                 'polID': '64509',
                 'polNum': '14562'},
                {'ballotOrder': 3,
                 'candidateID': '20410',
                 'first': 'Chris',
                 'last': 'Christie',
                 'party': 'GOP',
                 'polID': '60051',
                 'polNum': '14563'},
                {'ballotOrder': 4,
                 'candidateID': '20411',
                 'first': 'Ted',
                 'last': 'Cruz',
                 'party': 'GOP',
                 'polID': '61815',
                 'polNum': '14564'},
     

Bastante mejor

### Usando pdb

A grandes rasgos, hay dos situaciones en las que voy a usar pdb

Metiendo en mi código llamadas a [*pdb.set_trace()*](https://docs.python.org/3.5/library/pdb.html#pdb.set_trace) para que el intérprete pare (lo que serían puntos de parada de depuración)

```python

import pdb
def debug_this(i1, i2):
    result = i1
    pdb.set_trace() # Va a parar aqui
    for i in range(5):
        result += i2
    return result```
    
    

Entrando en una sesión del intérprete de Python y depurando explicitamente la función mediante una llamada a [*pdb.runcall*](https://docs.python.org/3.5/library/pdb.html#pdb.runcall)

```python

import pdb

# Ahora la función no tiene puntos de parada
def debug_this(i1, i2):
    result = i1
    for i in range(5):
        result += i2
    return result
    
# Pero ya la llamo yo
pdb.runcall(debug_this, 1, 1)```

En cualquiera de los dos casos, entraríamos en una sesión del depurador y podríamos ejecutar las instrucciones paso a paso, entre otros comandos. Los más populares que podríamos usar son:

#### Comando (l)ist 

Para ver las líneas de código donde nos encontramos

```python

(Pdb) l
  1  	def debug_this(i1, i2):
  2  ->	    result = i1
  3  	    for i in range(5):
  4  	        result += i2
  5  	    return result```

#### Comando (w)here

Para ver dónde nos encontramos dentro de la pila de llamadas (puede ser bastante profunda)

```python

(Pdb) w
  /usr/lib/python3.5/bdb.py(465)runcall()
-> res = func(*args, **kwds)
> <ipython-input-2-a2c915a1f1da>(2)debug_this()```

#### Comando (s)tep

Para entrar en el código de una función

```python
(Pdb) l
  1  	def foo():
  2  	    a = "hola"
  3  	    b = "bola"
  4  ->	    bar()
  5  	
[EOF]
(Pdb) s
--Call--
> <ipython-input-4-d5b067988c12>(1)bar()
-> def bar():
(Pdb) l
  1  ->	def bar():
  2  	    x = 2
  3  	    y = 3
  4  	    z = x + y
  5  	
[EOF]```


#### Comando (b)reak

Para añadir un punto de parada. Acepta dos modalidades:

* b [n]: Siendo n una línea del fichero actual (si estamos depurando una función en un fichero, por ejemplo). Si es una sesión de depuración en vivo, no cargando un fichero, simplemente indica el número de línea

* b [funcion]: Siendo *funcion* una función concreta. Añade un punto de parada en la primera línea evaluable de la función

```python
pdb.runcall(foo)
> <ipython-input-5-22ce54bbfd1f>(2)foo()
-> a = "hola"
(Pdb) l
  1  	def foo():
  2  ->	    a = "hola"
  3  	    b = "bola"
  4  	    bar()
  5  	
[EOF]
(Pdb) b bar
Breakpoint 2 at <ipython-input-4-d5b067988c12>:1```



#### Comando (c)ontinue

Para continuar hasta el siguiente punto de parada, o hasta el final del programa, si no hay ninguno más

```python
pdb.runcall(foo)
> <ipython-input-5-22ce54bbfd1f>(2)foo()
-> a = "hola"
(Pdb) l
  1  	def foo():
  2  ->	    a = "hola"
  3  	    b = "bola"
  4  	    bar()
  5  	
[EOF]
(Pdb) b bar
Breakpoint 2 at <ipython-input-4-d5b067988c12>:1
(Pdb) c
> <ipython-input-4-d5b067988c12>(2)bar()
-> x = 2
(Pdb) l
  1 B	def bar():
  2  ->	    x = 2
  3  	    y = 3
  4  	    z = x + y
  5  	
[EOF]```

#### Inspeccionar valores

En cualquier momento, podemos imprimir el valor de una variable para ver su contenido en ese momento.

```python
(Pdb) x
2```

Si quieres investigar un poco más, puedes probar a ejecutar los tests unitarios de

```python

python3 -m unittest tests.test_using_debugger```

### Rastreando errores

Vamos a intentar encontrar un problema en el código usando pdb. Ejecutemos

```python3

python3 -m unittest tests.test_chicago_result_loader.TestBrokenChicagoResultLoader```

Veremos una salida de error similar a ésta

```bash
F
======================================================================
FAIL: test_load (tests.test_chicago_result_loader.TestBrokenChicagoResultLoader)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/jorge/Dev/github/nicar2016-python-testing-debugging-exercises/tests/test_chicago_result_loader.py", line 27, in test_load
    self.assertEqual(alvarez['contest_code'], '0079')
AssertionError: '0790' != '0079'
- 0790
?    -
+ 0079
? +


----------------------------------------------------------------------
Ran 1 test in 0.012s

FAILED (failures=1)```

Por lo que se deduce del error, unos datos de un diccionario parecen erróneos, o desplazados con respecto al valor esperado.

El test ejecuta el método *BrokenChicagoResultsLoader.load*, que carga datos del fichero *tests/data/summary.txt*. En dicho fichero hay un registro por línea (es un fichero [TSV](https://en.wikipedia.org/wiki/Tab-separated_values) en realidad), y parece que hay algún problema leyendo el campo *contest_code*. 

<div class="alert alert-warning">__Ejercicio__: Investiga mediante una sesión de depuración cuáles son los valores esperados para ese campo, y cuáles son los valores que en realidad se están leyendo y porqué. Una vez identificado el error, arréglalo y vuelve a pasar los tests

<ul>
<li>__Pista__: En el método BrokenChicagoResultsLoader.load explica el formato del fichero</li>

<li>__Pista__: Fíjate en el nombre que causa el error. Tal vez te interese centrarte en ese nombre, generando una versión reducida del fichero para que te resulte más sencillo</li>
</ul>

</div>

In [6]:
# Esta celda da el estilo al notebook
from IPython.core.display import HTML
css_file = '../static/styles/style.css'
HTML(open(css_file, "r").read())